<h1><center>Semester Project</center></h1>


Over the past decades, the development of online scientific platforms like 
arXiv.org or JSTOR radically changed the way researchers access, browse, or read 
scientific articles. Yet, while observing the behavior of researchers in libraries and 
laboratories has become commonplace in the humanities, the computational study 
of digital research practices is only in its early days. This project aims at documenting 
the behavior of scientists on online platforms by making sense of the digital traces 
they generate while navigating.

During this project we are using the browsing logs of Gallica ( https://gallica.bnf.fr/ ) to make sense of user experience and identify patterns.

Project goals : 

* Parsing and sessionizing the log files in order to extract relevant sessions.
* Building an ontology based on the descriptors provided by the ARKs (Archival Resource Key, i.e. the metadata) of the documents (type of document, discipline, year, etc.)
* Using off-the-shelf Python libraries of topological data analysis (TDA) to identify the geometrical shapes of users’ paths through the ontology
* Building a typology of users’ behaviour on the platform by clustering the previously identified users’ paths

## Table Of Content:
[I. Exploring Gallica Logs](#first-bullet)

[II. Creating Sessions](#second-bullet)

[III. Word2vec Representation](#third-bullet)

[IV. Path Representation](#fourth-bullet)

<a class="anchor" id="first-bullet"></a><h2><center>I. Exploring Gallica Logs</center></h2> 

## 1. Importing logs

Logs have been provided by TODO, they span from a period of TODO.

In [43]:
import os
import numpy as np


# for now I manually extracted a single file to test
file  = open('res296.log','r',encoding="utf8")
lines = file.read().splitlines()

## 2. Exploring logs

In [125]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# convert lines from logs into pandas dataframe
lines_df = pd.DataFrame(lines)

In [45]:
lines_df.head()

,0
0,##320022e99796ca35dab7e63d48fd5e7##null##null#...
1,##e7fdec50f50253f6796d61b5382155f8##null##null...
2,##320022e99796ca35dab7e63d48fd5e7##null##null#...
3,##e7fdec50f50253f6796d61b5382155f8##null##null...
4,##320022e99796ca35dab7e63d48fd5e7##null##null#...


In [46]:
# we need to split each line into relevant metadata according to this example from Nouvellet et al.

![caption](log_example.png)

In [47]:
# first split, according to the example we split by ## to get ip, pays, ville and then date/requete/procole/code/taille/référant together
lines_df=lines_df[0].str.split('##', expand=True)

In [48]:
lines_df.head()

,0,1,2,3,4
0,,320022e99796ca35dab7e63d48fd5e7,null,null,"- - [03/Mar/2017:10:58:15 +0100] ""GET /ark:/12..."
1,,e7fdec50f50253f6796d61b5382155f8,null,null,"- - [03/Mar/2017:10:58:41 +0100] ""GET /ark:/12..."
2,,320022e99796ca35dab7e63d48fd5e7,null,null,"- - [03/Mar/2017:11:01:15 +0100] ""GET /ark:/12..."
3,,e7fdec50f50253f6796d61b5382155f8,null,null,"- - [03/Mar/2017:11:01:41 +0100] ""GET /ark:/12..."
4,,320022e99796ca35dab7e63d48fd5e7,null,null,"- - [03/Mar/2017:11:04:15 +0100] ""GET /ark:/12..."


### 2.1 Parsing HTTP requests

#### Experimenting with extracting data

In [49]:
# This is just experimental, TODO change with lazy eval regex when working on bigger data.

In [50]:
import re
import shutil
import requests
import xmltodict
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

In [51]:
'''
Function to query Gallica API and get metadata about a document from it's ark.
inputs: 
    'id': here id is the ARK of the document, that we extract from the request
Outputs:
     this function outputs the result of the API as a dictionary.

'''
def OAI(id):

    OAI_BASEURL = 'https://gallica.bnf.fr/services/OAIRecord?ark='

    url = "".join([OAI_BASEURL, id])
    #print(url)

    s = requests.get(url, stream=True)
    soup = BeautifulSoup(s.content,"lxml-xml")
    #print(soup)
    file = open('oai.xml', 'wb')
    file.write(soup.prettify().encode('UTF-8'))
    file.close()
    with open('oai.xml',encoding='UTF-8') as xml:
        doc = xmltodict.parse(xml.read())
        return doc


In [52]:
temp = pd.DataFrame()

In [53]:
# extracting the date of query
temp['Date']=lines_df.apply(lambda x: x[4].split("]")[0].split("[")[1] ,axis = 1)

In [54]:
# extracting request
temp['Request'] = lines_df.apply(lambda x: ' '.join(x[4].split("\"")[1].split(' ')[:2]),axis=1)

In [55]:
# extracting protocol, still need to look into it further
# do i really need to extract protocol at this point?
# commenting this for now until further research

# temp['protocol'] = lines_df.apply(lambda x: x[4].split("\"")[1].split(' ')[2], axis=1)

In [56]:
# extracting code
# try catch to avoid index errors, need to look into this further, what happens exactly there?
def trycode(value, default):
    try:
        return value[4].split("\"")[2].split(' ')[1]
    except (IndexError):
        return '-'
    return default


temp['Code'] = lines_df.apply(lambda x: trycode(x,'-'), axis=1)

In [57]:
# extracting length
# try catch to avoid index errors, need to look into this further, what happens exactly there?
def trylength(value, default):
    try:
        return value[4].split("\"")[2].split(' ')[2] 
    except (IndexError):
        return '-'
    return default

temp['Length'] = lines_df.apply(lambda x:  trylength(x,'-') , axis=1)

In [58]:
# extracting referant
temp['Referant'] = lines_df.apply(lambda x: x[4].split("\"")[3], axis=1)

In [60]:
# extracting ark name

#function to check if the request contains ark
def extract_ark(request):
    # capture everything between 12148 and / or between 12148 and . using regex
    ark = '-'
    ark = re.findall('(?<=12148/).+?(?=/)|(?<=12148/).+?(?=\.)', request)
    return ark


temp['Ark'] = temp.apply(lambda x: extract_ark(x['Request']), axis=1)
    

In [61]:
lines_df = lines_df.rename(columns={1:"IPAdress",2:"Country",3:"City",4:"Full_request"})

In [62]:
final_df = pd.concat([lines_df, temp],axis=1)

In [76]:
final_df[1195:1200]

,0,IPAdress,Country,City,Full_request,Date,Request,Code,Length,Referant,Ark
1195,,dd7e7fe210e8c18c7652e3f0e48700b2,France,Villemoustaussou,"- - [03/Mar/2017:18:04:59 +0100] ""GET /ark:/12...",03/Mar/2017:18:04:59 +0100,GET /ark:/12148/bpt6k1263204k.thumbnail,200,2291,http://gallica.bnf.fr/services/engine/search/s...,[bpt6k1263204k]
1196,,dd7e7fe210e8c18c7652e3f0e48700b2,France,Villemoustaussou,"- - [03/Mar/2017:18:04:59 +0100] ""GET /ark:/12...",03/Mar/2017:18:04:59 +0100,GET /ark:/12148/bpt6k5761916m.thumbnail,200,11323,http://gallica.bnf.fr/services/engine/search/s...,[bpt6k5761916m]
1197,,dd7e7fe210e8c18c7652e3f0e48700b2,France,Villemoustaussou,"- - [03/Mar/2017:18:04:59 +0100] ""GET /service...",03/Mar/2017:18:04:59 +0100,GET /services/ajax/extract/ark:/12148/bpt6k623...,200,1047,http://gallica.bnf.fr/services/engine/search/s...,[bpt6k6237463k]
1198,,dd7e7fe210e8c18c7652e3f0e48700b2,France,Villemoustaussou,"- - [03/Mar/2017:18:04:59 +0100] ""GET /ark:/12...",03/Mar/2017:18:04:59 +0100,GET /ark:/12148/bpt6k5495010c.thumbnail,200,9595,http://gallica.bnf.fr/services/engine/search/s...,[bpt6k5495010c]
1199,,dd7e7fe210e8c18c7652e3f0e48700b2,France,Villemoustaussou,"- - [03/Mar/2017:18:04:59 +0100] ""GET /service...",03/Mar/2017:18:04:59 +0100,GET /services/ajax/extract/ark:/12148/bpt6k553...,200,1065,http://gallica.bnf.fr/services/engine/search/s...,[bpt6k5535441w]


<a class="anchor" id="second-bullet"></a><h2><center>II. Creating Sessions</center></h2> 

#### 1. Creating a dataframe with IP and the difference in time between each connexion and the last

In [79]:
# Session: séquences de requêtes
# Regrouper même adresse IP => session se termine intervalle supérieur à 60 minutes entre deux requêtes. 
sessions_df = final_df.groupby('IPAdress').agg({'Ark':list,'Date':list})
sessions_df.head()

,Ark,Date
IPAdress,,
103e44bc19d6aac58db9a149c73e505b,[[]],[03/Mar/2017:18:12:04 +0100]
105781f3101367c473a91d52b6d4fd67,"[[bpt6k54673247], [bpt6k54673247], [bpt6k62397...","[03/Mar/2017:18:27:36 +0100, 03/Mar/2017:18:27..."
10907c8edc0b2702015e04f49a8204a2,"[[], [], [], [], [bpt6k6308044k], [bpt6k759364...","[03/Mar/2017:18:05:21 +0100, 03/Mar/2017:18:05..."
10915f6650d7b3ab000aafb953615c4e,"[[bpt6k33258628], [bpt6k3321225p], [bpt6k62553...","[03/Mar/2017:19:40:11 +0100, 03/Mar/2017:19:41..."
10dfc529d2b8f1a7ae6f94229848fbf,"[[bpt6k4453214], [], [], [], [], [], [], [], [...","[03/Mar/2017:18:38:47 +0100, 03/Mar/2017:18:39..."


In [82]:
from datetime import datetime
'''
Function to calculate absolute value of minutes between two dates
inputs: 
    d1: first date
    d2: second date
Outputs:
    absolute value of minutes between d1 and d2

'''
def minutes_between(d1, d2):
    d1 = datetime.strptime(d1, "%d/%b/%Y:%H:%M:%S")
    d2 = datetime.strptime(d2, "%d/%b/%Y:%H:%M:%S")
    return abs(((d2 - d1)).total_seconds() // 60.0)

In [85]:
time_beginning = "01/Jan/0001:01:01:01 +0100"
time_end = "01/Jan/3000:01:01:01 +0100"
sessions_df['date_1'] = sessions_df.apply(lambda x: [time_beginning]+x['Date'], axis = 1)
sessions_df['date_2'] = sessions_df.apply(lambda x: x['Date']+[time_end],axis=1)

In [91]:
'''
Function to calculate the difference between two zipped lists
'''
def calculate_difference_zipped_list(lst):
    new_lst = []
    for e in lst:
        if (e[0]==time_beginning):
            new_lst.append(999)
        elif (e[1]==time_end):
            new_lst.append(999)
        else:
            new_lst.append(minutes_between(e[0][:-6], e[1][:-6]))
    return new_lst
        
    

In [92]:
# this contains the ip adress and the zipped version of date_1,date_2
from collections import deque
IP_and_sessions = sessions_df.apply(lambda x: deque(calculate_difference_zipped_list(list(zip(x['date_1'],x['date_2'])))),axis=1)

In [98]:
# IP and the difference in time between each connection and the last
IP_and_sessions[:2]

IPAdress
103e44bc19d6aac58db9a149c73e505b                                           [999, 999]
105781f3101367c473a91d52b6d4fd67    [999, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
dtype: object

#### 2. Enriching the original dataframe with previous connexion date column, this will be useful when we try to set a timeout to define a session

In [89]:
# index on ipadress/date
# queue 
final_df['previous_connexion_date']=final_df.apply(lambda x: IP_and_sessions[x['IPAdress']].popleft(),axis=1)

In [96]:
final_df.head(2)

,0,IPAdress,Country,City,Full_request,Date,Request,Code,Length,Referant,Ark,previous_connexion_date
0,,320022e99796ca35dab7e63d48fd5e7,null,null,"- - [03/Mar/2017:10:58:15 +0100] ""GET /ark:/12...",03/Mar/2017:10:58:15 +0100,GET /ark:/12148/bpt6k70211m,503,-,-,[],999.0
1,,e7fdec50f50253f6796d61b5382155f8,null,null,"- - [03/Mar/2017:10:58:41 +0100] ""GET /ark:/12...",03/Mar/2017:10:58:41 +0100,GET /ark:/12148/bpt6k70211m,503,-,-,[],999.0


#### 3. Assigning a session ID to each connection according to the set rules we have set above

In [100]:
'''
Function to query Gallica API and get metadata about a document from it's ark.
inputs: 
    'id': here id is the ARK of the document, that we extract from the request
Outputs:
     this function outputs the result of the API as a dictionary.

'''
session_id=0
def create_session(period):
    global session_id
    if(period>30):
        session_id += 1
    return session_id

In [102]:
final_df=final_df.sort_values(by=['IPAdress','Date'])
final_df['session_id'] = final_df.apply(lambda x: create_session(x['previous_connexion_date']),axis=1)

#### 4. Creating sessions by grouping by session ID and collecting all ARKs


In [103]:
sessions = final_df.groupby('session_id').agg({'Ark':list})

#### 5. Converting arks from sessions to document titles

In [113]:
'''
get titles from ARKs

Input: list of arks

Output: list of document titles

'''
def get_title_from_ark(l):
    temp = []
    for ark in l:  
        title = ''
        if(len(ark)>0):
            oai_result = OAI(ark[0]) 
            if(oai_result != None ):
                try:
                    title = oai_result.get('results').get('notice').get('record').get('metadata').get('oai_dc:dc').get('dc:title')
                except:
                    title = ''
                
        temp.append(title)
    return temp

In [121]:
'''
get titles from ARKs using caching, if a user opens the same document twice we do not do any API call

Input: list of arks

Output: list of document titles

'''
def get_title_from_ark_cache(l):
    temp = []
    cache = {}
    for ark in l:  
        title = ''
        # remembering that l is a list of list [[ark1],[ark2],[ark3]]
        # if ark is not empty
        if(len(ark)>0):
            # check if ark in cache
            if ark[0] in cache:
                temp.append(cache.get(ark))
                continue
            else:            
                oai_result = OAI(ark[0]) 
                if(oai_result != None ):
                    try:
                        title = oai_result.get('results').get('notice').get('record').get('metadata').get('oai_dc:dc').get('dc:title')
                        cache["ark"]=title
                    except:
                        title = ''

        temp.append(title)
    return temp

In [122]:
sessions_5 = sessions[:5]

In [126]:
sessions_5['document_titles_path'] = sessions_5.apply(lambda x: get_title_from_ark_cache(x['Ark']),axis = 1)

In [128]:
# testing on 5 sessions, API calls are taking a bit too much time so far.
# TODO: contact Gallica ?
# TODO: get more than titles?
sessions_5

,Ark,document_titles_path
session_id,,
1,[[]],[]
2,"[[bpt6k54673247], [bpt6k54673247], [bpt6k62397...","[Le Progrès, Le Progrès, L'Avenir de Souk-Ahra..."
3,"[[], [], [], [], [bpt6k6308044k], [bpt6k759364...","[, , , , Le Rire : journal humoristique, L'Act..."
4,"[[bpt6k33258628], [bpt6k3321225p], [bpt6k62553...",[La pratique administrative dans la fonction p...
5,"[[bpt6k4453214], [], [], [], [], [], [], [], [...",[Matériaux pour l'histoire primitive et nature...


#### 6. TODO converting arks to themes

<a class="anchor" id="third-bullet"></a><h2><center>III. Word2vec Representation</center></h2> 

In this part I will try to represent the arks in a 2D dimension using word2vec, each ark will be assigned a vector.

Word2vec model is trained directly from the requests.

Choice between 2 models : 
* Skip-gram: Model will take one word as input, but it will return multiple words as per window size.
* CBOW (Continuous Bag of Words): takes the context of each word as the input and tries to predict the word corresponding to the context. 


#### 1. Extract referant ark

#### 2. Creating corpus

#### 3. Inputing corpus to the model

#### 4. Training model

#### 5. Model results + representation

<a class="anchor" id="fourth-bullet"></a><h2><center>IV. Path representation </center></h2> 